In [18]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [19]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
device

device(type='cuda')

## Net construction

In [21]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

#batch_size=
#num_classes=
#learning_rate=
#num_epochs=
strides=[1,3,3]
vertical_dilation=[1,2,3]
blocks=[2,3,4]
num_filters=64
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
use_gpu = torch.cuda.is_available()

In [22]:
# only for mac
# device=torch.device('mps')

In [23]:
class Day_20cnn(nn.Module):
    def __init__(self):
        super(Day_20cnn, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=64,kernel_size=(5,3),padding=(7,1),stride=(3,1),dilation=(2,1)),
                                  nn.BatchNorm2d(64), # Accelerating Deep Network Training by Reducing Internal Covariate Shift, normliaze data/features
                                  nn.LeakyReLU(),
                                  nn.MaxPool2d(kernel_size=(2,1))
                                  )
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(5,3),padding=(2,1),stride=1),
                                  nn.BatchNorm2d(128),
                                  nn.LeakyReLU(),
                                  nn.MaxPool2d(kernel_size=(2,1))
                                  )
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(5,3),padding=(2,1),stride=1),
                                  nn.BatchNorm2d(256),
                                  nn.LeakyReLU(),
                                  nn.MaxPool2d(kernel_size=(2,1))
                                  )
        self.fc1 = nn.Sequential(nn.Dropout(p=0.5),
                                 nn.Linear(46080,2))
        
    def forward(self,x):
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.conv3(y)
        y = y.view(y.shape[0],-1)
        y = self.fc1(y)
        # y=self.sfm(y)
        return y

In [24]:
net_day20=Day_20cnn()
# device_ids = [0, 1]
# net_day20 = torch.nn.DataParallel(net_day20, device_ids=device_ids)
net_day20.cuda()
from torchsummary import summary
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#use_gpu = torch.cuda.is_available()
#device = torch.device("cuda" if use_gpu else "cpu")
#print(device)

summary(net_day20, input_size=(1,64,60))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 24, 60]           1,024
       BatchNorm2d-2           [-1, 64, 24, 60]             128
         LeakyReLU-3           [-1, 64, 24, 60]               0
         MaxPool2d-4           [-1, 64, 12, 60]               0
            Conv2d-5          [-1, 128, 12, 60]         123,008
       BatchNorm2d-6          [-1, 128, 12, 60]             256
         LeakyReLU-7          [-1, 128, 12, 60]               0
         MaxPool2d-8           [-1, 128, 6, 60]               0
            Conv2d-9           [-1, 256, 6, 60]         491,776
      BatchNorm2d-10           [-1, 256, 6, 60]             512
        LeakyReLU-11           [-1, 256, 6, 60]               0
        MaxPool2d-12           [-1, 256, 3, 60]               0
          Dropout-13                [-1, 46080]               0
           Linear-14                   

## Date load

In [25]:
import torchvision
import os
import pandas as pd
import numpy as np
IMAGE_WIDTH={5:15, 20:60, 60:180}
IMAGE_HEIGHT={5:32, 20:64, 60:96}

def data_set(dir, year_index, in_size, out_size):
    year = year_index[0]
    img_path = os.path.join(dir,"monthly_20d",f"20d_month_has_vb_[20]_ma_{year}_images.dat")
    label_path = os.path.join(dir,"monthly_20d",f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather")
    image = np.memmap(img_path,dtype=np.uint8,mode='r').reshape(-1,IMAGE_HEIGHT[in_size],IMAGE_WIDTH[out_size])
    label = pd.read_feather(label_path)

    for year in year_index[1:]:
        img_path = os.path.join(dir,"monthly_20d",f"20d_month_has_vb_[20]_ma_{year}_images.dat")
        label_path = os.path.join(dir,"monthly_20d",f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather")
        img = np.memmap(img_path,dtype=np.uint8,mode='r').reshape(-1,IMAGE_HEIGHT[in_size],IMAGE_WIDTH[out_size])
        lbl = pd.read_feather(label_path)
        image = np.concatenate((image,img),axis=0)
        label = pd.concat([label,lbl])
    return image, label

def sampling(image, label):
    len_df = len(label)
    random_index = np.random.permutation(len_df)
    train_index = random_index[:int(0.7*len_df)]
    train_image = image[train_index]
    # print(label.index)
    # print(train_index)
    train_label = label.iloc[train_index,:]
    validation_index = random_index[int(0.7*len_df):]
    validation_image = image[validation_index]
    validation_label = label.iloc[validation_index,:]
    return train_image, train_label, validation_image,validation_label

In [26]:
# we train and validate each model only once using data from 1993 to 2000, in
# which 70% of the sample are randomly selected for training and the remaining 30% for validation.
# train - validation
# img_dir = "/Users/evan_wang/Library/CloudStorage/OneDrive-Microsoft/code/python/pytorch/data"
img_dir = "/home/ywangtb/ai_fintech"
train_index = np.arange(1993, 2001, 1)
# train = True
in_size = 20
out_size = 20
image, label = data_set(img_dir, train_index, in_size, out_size)
train_image, train_label, validation_image,validation_label = sampling(image, label)

In [27]:
# test
test_index = np.arange(2001, 2002, 1)
train = False
in_size = 20
out_size = 20
test_image, test_label = data_set(img_dir, test_index, in_size, out_size)

In [28]:
class Stock(torch.utils.data.Dataset):
    def __init__(self, image, label,transform):
        self.image = image
        self.label = label
        self.transform = transform
        self.pre_process()
    def __len__(self):
        return len(self.label)
    def __getitem__(self, index):
        image = self.transform(self.image[index])
        return image, self.label.iloc[index, -1]
    def pre_process(self):
        self.label['Label_20d'] = self.label['Ret_20d']
        # self.label['Label_20d'] = self.label['Ret_5d']
        self.label.loc[self.label['Label_20d']<=0, 'Label_20d']=0
        self.label.loc[self.label['Label_20d']>0, 'Label_20d']=1
        self.label.loc[pd.isnull(self.label['Label_20d']), 'Label_20d']=0


In [29]:
mytransform = torchvision.transforms.ToTensor()
train_data = Stock(train_image, train_label, mytransform)
validation_data = Stock(validation_image, validation_label, mytransform)
test_data = Stock(test_image, test_label, mytransform)

/tmp/ipykernel_586460/3141858913.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.label['Label_20d'] = self.label['Ret_20d']
/tmp/ipykernel_586460/3141858913.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.label['Label_20d'] = self.label['Ret_20d']


In [30]:
train_loader=torch.utils.data.DataLoader(dataset=train_data, batch_size=128,shuffle=True,num_workers=2)
validation_loader=torch.utils.data.DataLoader(dataset=validation_data, batch_size=128,shuffle=True,num_workers=2)
test_loader=torch.utils.data.DataLoader(dataset=test_data, batch_size=128,shuffle=True,num_workers=2)
# loaders = {'train': train_loader, 'validation': validation_loader}
loaders = {'train': train_loader, 'validation': validation_loader, 'test': test_loader}

In [31]:
# examples = enumerate(data_loader)
# batch_idx, (example_data, example_targets) = next(examples)

In [32]:
# import matplotlib.pyplot as plt
# plt.imshow(example_data[0].numpy(), cmap='gray')

# Train the model

In [33]:
import os
import re
import time


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=1):
    batch_size = target.size(0)
    _, pred = output.topk(topk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    res.append(correct.sum()/batch_size*100)
    # print(res)
    return res

def train_model(model, criterion, optimizer, log_saver):

    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    model = model.cuda()
    model.apply(init_weights)

    since = time.time()
    steps = 0
    epoch = 1
    stopping = 0
    iter = 0
    # we use early stopping to halt training once the
    # validation sample loss function fails to improve for two consecutive epochs.
    while(stopping < 2):
    # while(iter == 0):
        iter += 1
        
        print('Epoch {}'.format(epoch))
        print('-' * 30)
        for phase in ['train', 'validation']: # may need consider validation to tune parameter

            loss_meter = AverageMeter()
            acc_meter = AverageMeter()
            # margin_error_meter = AverageMeter()

            if phase == 'train':
                print('train','-' * 24)
                model.train(True)
            else:
                print('validation','-' * 19)
                model.train(False)

            current = 0

            for i, data in enumerate(loaders[re.findall('[a-zA-Z]+',
                                                        phase)[0]]):
                inputs, labels = data
                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                # print(labels)
                labels = labels.long()

                # print(labels)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    steps += 1

                N = outputs.size(0)
                # print(N)
                loss_meter.update(loss.data.item(), N)
                acc_meter.update(
                    accuracy(outputs.data, labels.data)[-1].item(), N)
                # if phase == 'train':
                current += len(inputs)
                if i % 500 == 499:
                    print(f"batch: {i+1} loss: {loss_meter.avg:>7f}  [{current:>5d}/{len(loaders[re.findall('[a-zA-Z]+',phase)[0]].dataset):>5d}]")

            epoch_loss = loss_meter.avg
            epoch_error = 1 - acc_meter.avg / 100

            if phase == 'train':
                log_saver['train_loss'].append(epoch_loss)
                log_saver['train_error'].append(epoch_error)

            elif phase == 'validation':

                log_saver['test_loss'].append(epoch_loss)
                log_saver['test_error'].append(epoch_error)

                if iter > 2 and (log_saver['test_loss'][-1] >= log_saver['test_loss'][-2]):
                    stopping += 1
                else:
                    stopping = 0

            print(
                f'-- {phase} loss: {epoch_loss:.4f}; error: {epoch_error:.4f}'
            )

        epoch += 1

        # if epoch % 10 == 0:
    print('Saving..')
    torch.save(model,'save.pt')


    time_elapsed = time.time() - since
    print(
        f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s'
    )
    # print("Now print the test outcome")
    # phase = "test"
    # inputs, labels = loaders[phase]
    # if use_gpu:
    # # inputs = inputs.to(device)
    # # labels = labels.to(device)
    #     inputs = inputs.cuda()
    #     labels = labels.cuda()

    # optimizer.zero_grad()

    # outputs = model(inputs)
    # _, preds = torch.max(outputs.data, 1)
    # labels = labels.long()
    # loss = criterion(outputs, labels)

    # epoch_loss = loss.data.item()
    # epoch_error = 1 - accuracy(outputs.data, labels.data)[-1].item() / 100
    # print(f'{phase} loss: {epoch_loss:.4f}; error: {epoch_error:.4f}')

    return model, log_saver


In [34]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net_day20.parameters(), lr=0.0001,betas = (0.9,0.99))
# optimizer = optim.SGD(net_day20.parameters(), lr=0.00001, momentum=0.9) # use adam
# device_ids = [0, 1]
# optimizer = nn.DataParallel(optimizer, device_ids=device_ids)
# The log for recording train (test) loss and errors.
log = {
    # 'num_params': [],
    'train_loss': [],
    'train_error': [],
    'test_loss': [],
    'test_error': []
}
model, log = train_model(
    net_day20, criterion, optimizer, log)

Epoch 1
------------------------------
train ------------------------
batch: 500 loss: 1.030984  [64000/555113]
batch: 1000 loss: 0.949619  [128000/555113]
batch: 1500 loss: 0.896092  [192000/555113]
batch: 2000 loss: 0.864153  [256000/555113]
batch: 2500 loss: 0.839929  [320000/555113]
batch: 3000 loss: 0.823541  [384000/555113]
batch: 3500 loss: 0.810108  [448000/555113]
batch: 4000 loss: 0.798978  [512000/555113]
-- train loss: 0.7931; error: 0.4921
validation -------------------
batch: 500 loss: 0.722041  [64000/237906]
batch: 1000 loss: 0.721535  [128000/237906]
batch: 1500 loss: 0.721959  [192000/237906]
-- validation loss: 0.7220; error: 0.5063
Epoch 2
------------------------------
train ------------------------
batch: 500 loss: 0.713485  [64000/555113]
batch: 1000 loss: 0.712225  [128000/555113]
batch: 1500 loss: 0.710884  [192000/555113]
batch: 2000 loss: 0.710420  [256000/555113]
batch: 2500 loss: 0.709255  [320000/555113]
batch: 3000 loss: 0.708374  [384000/555113]
batch: 3

In [55]:
x = torch.from_numpy(test_image[:]/255).reshape(len(test_image),1,64,60).cuda().float()


In [ ]:
# log_pd = pd.DataFrame(log)
# log_pd.to_csv('log_pd.csv')

In [ ]:
# examples = enumerate(validation_loader)
# batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
# print(example_targets[-1])

In [ ]:
torch.save(model,'save.pt')
img_dir = "/content/drive/My Drive"
model=torch.load(os.path.join(img_dir,"20cnn.pt"), map_location='cpu')

model.eval()


from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, deprocess_image, preprocess_image
import matplotlib.pyplot as plt


import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
im_tensor = example_data
im_tensor=im_tensor.reshape(len(example_data),1,64,60)
input_tensor = im_tensor[-1]
input_tensor = input_tensor.reshape(1,1,64,60)

rgb_img = cv2.cvtColor(np.array(example_data[-1][0]).astype(np.uint8), cv2.COLOR_GRAY2BGR)
rgb_img = np.float32(rgb_img)



target_layers = [[model.conv1],[model.conv2],[model.conv3]]
plt.figure(figsize=(8,8), dpi=80)
plt.figure(1)


img = np.array(example_data[-1][0]).astype(np.uint8)
example_data3=Image.fromarray(img).convert( "L" ) # numpy 转 image类
ax1 = plt.subplot(2,2,1)
ax1.imshow(example_data3)
plt.title("Original OHLC")
for id, layer in enumerate(target_layers):
    cam = GradCAM(model=model, target_layers=layer)
    cam = cam(input_tensor=input_tensor, targets=None)

    grayscale_cam = cam[0,:]

    visualization = show_cam_on_image(np.array(rgb_img).astype(np.uint8) , grayscale_cam, use_rgb=False)
    ax1 = plt.subplot(2,2,id+2)
    ax1.imshow(visualization)
    plt.title("Layer {} OHLC".format(id+1))
plt.show()